In [2]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [3]:
# Read the .csv file
df_nissan = pd.read_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\\raw_datasets\\nissanR.csv")

<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_9052\2543346144.py:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
  df_nissan = pd.read_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\\raw_datasets\\nissanR.csv")


In [4]:
# Print the head
df_nissan.head(2)

,ModType,FS1 Hardware,HITACHI PN,Hitachi Cache,NISSAN PN,Last Grp,Nissan Cache,EEPROM \nTYPE,23703 BLANK,23710 / 23703 PN,Example VIN,Year,Make & Model,Engine,Trans & Emissions,EPC Supression
0,NS-H20,H20-39FM0,BEM172-300 A1,J6,NEC009-075,??,??,NaN,NaN,23703-9FM0A,1N6AA0ECXEN504194,2014,Nissan Titan,5.6L VK56DE,"AT, FED",NaN
1,NS-H20,H20-39FM0,BEM172-300 A1,J6,NEC011-018,??,??,NaN,NaN,23703-9FM0A,5N1AA0NC3FN604105,2015,Nissan Armada,5.6L VK56DE,"AT, FED",NaN


In [5]:
# Get the index with empty rows
df_nissan_empty_rows_index = df_nissan[df_nissan['ModType'].isna() == True].index

In [6]:
# Drop the empty rows
df_nissan_drop_empty = df_nissan.drop(df_nissan_empty_rows_index)

In [7]:
# Drop the repeated columns
df_nissan_repeated_cols_index = df_nissan_drop_empty[df_nissan_drop_empty['ModType'] == 'ModType'].index

In [8]:
# Drop the repeated cols
df_nissan_repeated_cols_dropped = df_nissan_drop_empty.drop(df_nissan_repeated_cols_index)

In [9]:
# Remove rows where all cols have null (NaN) values,
# keeping only rows with at least one valid (non-null) entry.
df_nissan_rm_null_rows = df_nissan_repeated_cols_dropped[~df_nissan_repeated_cols_dropped.isnull().all(axis=1)]

In [10]:
# Keep only rows where more than one col has a non-null value.
# This removes rows with only a single non-null entry, ensuring each row has multiple valid data points.
df_nissan_rm_one_null_rows = df_nissan_rm_null_rows[df_nissan_rm_null_rows.notnull().sum(axis=1) != 1]

In [11]:
# Show cols
df_nissan_rm_one_null_rows.columns

Index(['ModType', 'FS1 Hardware', 'HITACHI PN', 'Hitachi Cache', 'NISSAN PN',
       'Last Grp', 'Nissan Cache', 'EEPROM \nTYPE', '23703 BLANK',
       '23710 / 23703 PN', 'Example VIN', 'Year', 'Make & Model', 'Engine',
       'Trans & Emissions', 'EPC Supression'],
      dtype='object')

# Cols to keep:
- FS1 Hardware 
- ModType
- HITACHI PN
- NISSAN PN 
- 23710 / 23703 PN

In [12]:
# List with cols to keep
cols_to_keep = [
    "FS1 Hardware",
    "ModType",
    "HITACHI PN",
    "NISSAN PN", 
    "23710 / 23703 PN"
]

In [13]:
# Create a df with needed cols
df_nissan_needed_cols = df_nissan_rm_one_null_rows[cols_to_keep]

In [14]:
# Rename 'Hardware' col
df_nissan_rn_hdw_col = df_nissan_needed_cols.rename(columns={"FS1 Hardware": "hardware"})

In [15]:
def fill_null_rows(df):
    '''
    Iterate over all columns and fill out all null values.

    Parameters: 
        df (DataFrame): main df.

    Returns:
        df: Return a df with the null values filled out
    '''
    for col in df.columns:
        df[col] = df[col].fillna("Not Available")

    return df

In [16]:
# Function to remove the whitespaces
def remove_whitespaces(df):
    # Loop to iterate over all cols
    for col in df.columns:
        # Remove the writespaces
        df[col] = df[col].str.strip()

    return df

In [17]:
# Function to split multiple part numbers originally in the same line to one row each
def split_part_numbers(df, col):

    # Call the function to remove the whitespaces
    df_rm_spaces = remove_whitespaces(df)

    df_copy = df_rm_spaces.copy()

    # Create a list with the items in each row 
    df_copy[col] = df_copy[col].str.split(",")
    # Explode the items in different rows each and keep the info from the other rows
    df_splitted = df_copy.explode(col)
    # Return the df with the exploded part numbers
    return df_splitted

In [18]:
# Call the function to fill out null values
df_nissan_fill_nulls = fill_null_rows(df_nissan_rn_hdw_col)

In [19]:
# Show cols
df_nissan_fill_nulls.columns

Index(['hardware', 'ModType', 'HITACHI PN', 'NISSAN PN', '23710 / 23703 PN'], dtype='object')

### Call the function to the explode all items of the cols

In [20]:
df_nissan_expl_hdw_col = split_part_numbers(df_nissan_fill_nulls, "hardware")

In [21]:
df_nissan_expl_modtype_col = split_part_numbers(df_nissan_expl_hdw_col, 'ModType')

In [22]:
df_nissan_expl_hitachiPn_col = split_part_numbers(df_nissan_expl_modtype_col, 'HITACHI PN')

In [23]:
df_nissan_expl_nissanPn_col = split_part_numbers(df_nissan_expl_hitachiPn_col, 'NISSAN PN')

In [24]:
df_nissan_expl_numberPn_col = split_part_numbers(df_nissan_expl_nissanPn_col, '23710 / 23703 PN')

## Standardizing Part Number Dataset Columns
Each part number dataset contains columns with different names because they originate from different manufacturers. One of the goals of cleaning these datasets is to create a standard format that allows for the identification of the hardware part number when inputting other part numbers, such as software part numbers or system part numbers.

Since the identification of the hardware part number will be performed by inputting multiple part numbers at once (via a .csv file), it is necessary — except for the hardware column — to rename the other columns to a standard format: `pnNum` (e.g., `pn1`, `pn2`, etc.). This ensures consistency and enables automated matching and processing across datasets from various manufacturers.

In [25]:
def rename_col_names(df):
    '''
    Except the column hardware, the function standardize the names for pnNumber (pn1, pn2).
    Needed due to each part number dataset has different column names, so that 

    Parameters:
    df (DataFrame): main df.
    col_list (list): List of original column names.

    Returns
    df: Return the main df with the renamed cols

    '''

    # List with the col names
    col_name_list = list(df.columns) 
    
    # Dict to append the old names (key): new names (values) 
    new_cols_dict = {}

    # Count the items under the col list
    col_num = len(col_name_list)
    # Loop to iterate over the list
    for col in col_name_list:
        # Condition to define when lower the col name and when rename it
        if col == "hardware":
            new_cols_dict[col] = col.lower()
        else:
            # Convert the string to number to sum with the stirng pn and
            # append as value to the dict
            new_cols_dict[col] = "pn" + str(col_num)
            # Return the number from string to int
            col_num = int(col_num)
        
        # Subtract one to create the next col name
        num_col = col_num - 1
        col_num = num_col

    # Return the main df with the cols remanes
    return df.rename(columns=new_cols_dict)

In [26]:
# Rename all cols
df_nissan_rn_cols = rename_col_names(df_nissan_expl_numberPn_col)

In [27]:
# Check df info
df_nissan_rn_cols.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2748 entries, 0 to 2796
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   hardware  2748 non-null   object
 1   pn4       2748 non-null   object
 2   pn3       2748 non-null   object
 3   pn2       2748 non-null   object
 4   pn1       2748 non-null   object
dtypes: object(5)
memory usage: 128.8+ KB


In [63]:
# Export to .csv file
df_nissan_rn_cols.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\data_cleaned\pn-nissan.csv", index=False)

<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_9272\339766686.py:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
  df_nissan_rn_cols.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\data_cleaned\pn-nissan.csv", index=False)


## Database Import Documentation

After cleaning the data and preparing the DataFrame as needed, the data will be imported into a database. The functions below are intended to connect to the database and import the datasets.

In [28]:
def import_datasets_to_db(df, df_name=str):

    '''
    Import the df cleaned to the db

    Parameters: 
        df (DataFrame): main df to import to the db.
        df_name: string to label the df into the db.

    Returns:
        Import the df to the db. 
    '''
    # Setting up the connection with the PosgreSQL
    dbname="prescreen_diag_data_api"
    user="postgres"
    password="shakey-10"
    host="localhost"
    port="5432"

    # String connection for SQLAlchemy (using psycopg2 as driver)
    engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

    return df.to_sql(df_name, engine, if_exists='replace', index=False)


In [29]:
# Call the function to import the df cleaned to the postgreSQLdb
import_datasets_to_db(df_nissan_rn_cols, 'part_numbers_nissan')

748